In [15]:

from dataclasses import dataclass
from typing import List, Dict, Any
import numpy as np


In [16]:
# Basic Structures


@dataclass
class Asset:
    price: float
    growth_rate: float

    def evolve_end_of_month(self, scenario: dict):
        # Placeholder logic (e.g. apply growth)
        pass

    def scale(self, reserve: float):
        # Placeholder logic
        pass

    def sell(self, expense: float):
        # Placeholder logic
        pass

@dataclass
class Liability:
    cost: float

    def evolve_end_of_month(self, scenario: dict):
        # Placeholder logic
        pass


# Portfolio Containers

@dataclass
class PortfolioAsset:
    assets: List[Asset]

    def get_sale_value(self) -> float:
        return sum(asset.price for asset in self.assets)

@dataclass
class PortfolioLiability:
    liabilities: List[Liability]

    def get_expenses(self) -> float:
        return sum(liability.cost / 12 for liability in self.liabilities)


# Reserve Adjustment Logic


def decrease_reserve(reserve: float, amount: float) -> float:
    return reserve - amount * 0.1  # Placeholder step

def increase_reserve(reserve: float, amount: float) -> float:
    return reserve + abs(amount) * 0.1


# Initialization


def initialize_portfolio_assets() -> PortfolioAsset:
    return PortfolioAsset(assets=[Asset(price=100.0, growth_rate=0.03)])

def initialize_portfolio_liabilities() -> PortfolioLiability:
    return PortfolioLiability(liabilities=[Liability(cost=1200.0)])


# Simulation Logic


def get_last_reserve_value(scenario, portfolio_assets, portfolio_liabilities) -> float:
    n_months = 720
    for _ in range(n_months):
        for asset in portfolio_assets.assets:
            asset.evolve_end_of_month(scenario)
        for liability in portfolio_liabilities.liabilities:
            liability.evolve_end_of_month(scenario)

        expense = portfolio_liabilities.get_expenses()
        for asset in portfolio_assets.assets:
            asset.sell(expense)

    return portfolio_assets.get_sale_value()

def get_best_reserve(scenario, portfolio_assets, portfolio_liabilities, initial_reserve) -> Dict[str, Any]:
    max_iterations = 50
    reserve = portfolio_assets.get_sale_value()

    for i in range(max_iterations):
        if i > 0:
            for asset in portfolio_assets.assets:
                asset.scale(reserve)

        last_reserve_value = get_last_reserve_value(scenario, portfolio_assets, portfolio_liabilities)

        if np.isclose(last_reserve_value, 0.0):
            return {"status": "Success", "reserve": reserve}

        if last_reserve_value > 0:
            reserve = decrease_reserve(reserve, last_reserve_value)
        else:
            reserve = increase_reserve(reserve, last_reserve_value)

    return {"status": "Fail", "reserve": reserve}

def run_scenarios(scenarios: List[Dict[str, Any]]) -> List[Dict[str, Any]]:
    best_reserves = []
    for scenario in scenarios:
        portfolio_assets = initialize_portfolio_assets()
        portfolio_liabilities = initialize_portfolio_liabilities()
        initial_reserve = portfolio_assets.get_sale_value()
        best_reserve = get_best_reserve(scenario, portfolio_assets, portfolio_liabilities, initial_reserve)
        best_reserves.append(best_reserve)
    return best_reserves


# Scenario


scenarios = [{"interest_rate_adjustment": 0.0}]
results = run_scenarios(scenarios)
print(results)


[{'status': 'Fail', 'reserve': -400.0}]
